<a href="https://colab.research.google.com/github/BA7164bb/Crop-Classification/blob/main/irrigated_rainfed_wheat_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.3/631.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.4/837.4 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


In [ ]:
#  Libraries
import ee
import geemap
import pandas as pd
import geopandas as gpd
import ast
import json

In [ ]:
# Authenticate Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-bbahmanabadi')

In [ ]:
shapefile_path_cities = "/content/drive/MyDrive/maroon_Basin_cities_Shahrestan.shp"
gdf_cities = gpd.read_file(shapefile_path_cities).to_crs("EPSG:4326")
cities_geojson = json.loads(gdf_cities.to_json())
cities_fc = geemap.geojson_to_ee(cities_geojson)

In [ ]:
shapefile_path_rivers = "/content/drive/MyDrive/khuzestan/Water-ways-Maroon.shp"
gdf_rivers = gpd.read_file(shapefile_path_rivers).to_crs("EPSG:4326")
rivers_geojson = json.loads(gdf_rivers.to_json())
rivers_fc = geemap.geojson_to_ee(rivers_geojson)

In [ ]:
def load_csv_as_features(path):
    df = pd.read_csv(path)
    df.columns = df.columns.str.lower()
    df['.geo'] = df['.geo'].apply(ast.literal_eval)
    df['longitude'] = df['.geo'].apply(lambda x: x['coordinates'][0])
    df['latitude'] = df['.geo'].apply(lambda x: x['coordinates'][1])
    return [ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]), {'class': int(row['class'])}) for _, row in df.iterrows()]

csv_paths = {
    'wheat': '/content/drive/MyDrive/wheat_export.csv',
    'alfalfa': '/content/drive/MyDrive/Alfalfa_export.csv',
    'canola': '/content/drive/MyDrive/canola_export.csv',
    'bean': '/content/drive/MyDrive/Bean_export.csv',
    'dates': '/content/drive/MyDrive/Dates_export.csv',
    'soil': '/content/drive/MyDrive/soil_export.csv',
    'waterboundary': '/content/drive/MyDrive/WaterBoundary_export.csv'
}

all_features = []
for name, path in csv_paths.items():
    all_features.extend(load_csv_as_features(path))
training_fc = ee.FeatureCollection(all_features)

In [ ]:
# Load Sentinel-2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
    .filterBounds(cities_fc)\
    .filterDate("2023-03-01", "2023-03-31")\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))\
    .median().clip(cities_fc)

def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    evi = image.expression('2.5*((NIR-RED)/(NIR+6*RED-7.5*BLUE+1))', {
        'NIR': image.select('B8'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}).rename('EVI')
    savi = image.expression('((NIR-RED)/(NIR+RED+0.5))*1.5', {
        'NIR': image.select('B8'), 'RED': image.select('B4')}).rename('SAVI')
    wwi = image.expression('(B8 - B11)/(B8 + B11)', {'B8': image.select('B8'), 'B11': image.select('B11')}).rename('WWI')
    lai = image.expression('3.618 * NDVI - 0.118', {'NDVI': ndvi}).rename('LAI')
    s2rep = image.expression('705 + 35 * ((RE1 + RE3)/2 - RE2) / ((RE1 - RE2)+1e-10)', {
        'RE1': image.select('B5'), 'RE2': image.select('B6'), 'RE3': image.select('B7')}).rename('S2REP')
    wdvi = image.expression('NIR - 1.0 * RED', {'NIR': image.select('B8'), 'RED': image.select('B4')}).rename('WDVI')
    mtci = image.expression('(B6 - B5) / ((B5 - B4)+1e-10)', {
        'B6': image.select('B6'), 'B5': image.select('B5'), 'B4': image.select('B4')}).rename('MTCI')
    gcc = image.expression('G / (R + G + B + 1e-10)', {
        'R': image.select('B4'), 'G': image.select('B3'), 'B': image.select('B2')}).rename('GCC')
    rendvi = image.normalizedDifference(['B8', 'B5']).rename('RENDVI')
    rvi = image.expression('NIR / (RED + 1e-10)', {'NIR': image.select('B8'), 'RED': image.select('B4')}).rename('RVI')
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    lswi = image.normalizedDifference(['B8', 'B11']).rename('LSWI')

    return image.addBands([ndvi, gndvi, evi, savi, wwi, lai, s2rep,
                           wdvi, mtci, gcc, rendvi, rvi, ndre, lswi])

image_with_indices = add_indices(s2)

In [ ]:
# Sentinel-1
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")\
    .filterBounds(cities_fc)\
    .filterDate("2023-03-01", "2023-03-31")\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .mean().clip(cities_fc)
vv = s1.select('VV').rename('VV')
vh = s1.select('VH').rename('VH')
vv_vh_ratio = vv.divide(vh).rename('VV_VH_Ratio')

combined_image = image_with_indices.addBands([vv, vh, vv_vh_ratio])

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'NDVI', 'GNDVI', 'EVI', 'SAVI', 'WWI', 'LAI',
         'S2REP', 'WDVI', 'MTCI', 'GCC', 'RENDVI', 'RVI', 'NDRE', '' 'VV', 'VH', 'VV_VH_Ratio']

In [ ]:
training = combined_image.select(bands).sampleRegions(collection=training_fc, properties=['class'], scale=10)
training = training.randomColumn('random', seed=100)
training_split = training.filter(ee.Filter.lt('random', 0.7))
testing_split = training.filter(ee.Filter.gte('random', 0.7))

classifier = ee.Classifier.smileRandomForest(100).train(features=training_split, classProperty='class', inputProperties=bands)
classified = combined_image.select(bands).classify(classifier)
wheat_mask = classified.eq(0)

In [ ]:
# LULC Layers
# LGRIP30: 1 = irrigated, 2 = rainfed
lgrip = ee.ImageCollection("projects/sat-io/open-datasets/GFSAD/LGRIP30").mosaic()
gfsad_irrigated = lgrip.eq(1)
gfsad_rainfed = lgrip.eq(2)

# Dynamic World: crop probability > 0.5
dw_prob = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")\
    .filterDate("2023-03-01", "2023-03-31")\
    .select('crops').mean()
dw_crops = dw_prob.gt(0.5)

# ESA WorldCover: 40 = cropland, 50 = built-up, 60 = bare/sparse
esa = ee.Image("ESA/WorldCover/v100/2020").select('Map')
esa_crops = esa.eq(40)
non_crop_mask = esa.eq(50).Or(esa.eq(60)).Or(esa.eq(80))  # built-up, bare, water

# NDVI Filter (vegetation greenness)
ndvi = combined_image.select('NDVI')
veg_mask = ndvi.gt(0.4)

# Fusion logic
fusion_irrigated_sources = gfsad_irrigated.add(esa_crops).add(dw_crops).add(veg_mask)
fused_irrigated = wheat_mask.And(fusion_irrigated_sources.gte(2)).And(non_crop_mask.Not())

rainfed_score = gfsad_rainfed.add(esa_crops.Not()).add(dw_crops.Not()).add(ndvi.gt(0.3))
fused_rainfed = wheat_mask.And(rainfed_score.gte(2)).And(fused_irrigated.Not())


In [ ]:
# Compute Area
def compute_area(mask):
    area_img = mask.multiply(ee.Image.pixelArea()).rename('area')
    stats = area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=cities_fc.geometry(),
        scale=30,
        maxPixels=1e13
    )
    return ee.Number(stats.get('area')).divide(10000)

print("Irrigated Wheat Area (ha):", compute_area(fused_irrigated).getInfo())
print(" Rainfed Wheat Area (ha):", compute_area(fused_rainfed).getInfo())

Irrigated Wheat Area (ha): 85893.09167342649
 Rainfed Wheat Area (ha): 13770.222261583107


In [ ]:
'''
# ------------------ Step 9: Area Per City ------------------
def area_per_city(mask, fc):
    area_img = mask.multiply(ee.Image.pixelArea()).rename('area')

    def compute(f):
        area = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=f.geometry(),
            scale=10,
            maxPixels=1e13
        ).get('area')
        return f.set('area_ha', ee.Number(area).divide(10000))

    return fc.map(compute)

rainfed_per_city = area_per_city(fused_rainfed, cities_fc)
irrigated_per_city = area_per_city(fused_irrigated, cities_fc)

rainfed_gdf = gpd.GeoDataFrame.from_features(geemap.ee_to_geojson(rainfed_per_city))
irrigated_gdf = gpd.GeoDataFrame.from_features(geemap.ee_to_geojson(irrigated_per_city))

print("\U0001F33E Rainfed Wheat Area per City (ha):")
print(rainfed_gdf[['SHAHRESTAN', 'area_ha']])

print("\U0001F4A7 Irrigated Wheat Area per City (ha):")
print(irrigated_gdf[['SHAHRESTAN', 'area_ha']])
'''

In [ ]:
# Visualizing
Map = geemap.Map()
Map.centerObject(cities_fc, 10)

Map.addLayer(s2, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB')
Map.addLayer(fused_irrigated.selfMask(),{'palette': ['#00BFFF']}, 'Irrigated Wheat')
Map.addLayer(fused_rainfed.selfMask(), {'palette': ['#FFA500']}, 'Rainfed Wheat')
Map.addLayer(cities_fc.style(color='black', fillColor='00000000', width=2), {}, 'Basin Boundary')

# Add Labels
label_gdf_cities = gdf_cities.copy()
label_gdf_cities['geometry'] = label_gdf_cities.centroid
label_gdf_cities['x'] = label_gdf_cities.geometry.x
label_gdf_cities['y'] = label_gdf_cities.geometry.y

Map.add_labels(label_gdf_cities, column='SHAHRESTAN', font_size='12pt', text_color='black', font_weight='bold', x='x', y='y')

label_gdf_rivers = gdf_rivers.copy()
label_gdf_rivers = label_gdf_rivers[label_gdf_rivers['name'].notna()].drop_duplicates(subset='name')
label_gdf_rivers['geometry'] = label_gdf_rivers.centroid
label_gdf_rivers['x'] = label_gdf_rivers.geometry.x
label_gdf_rivers['y'] = label_gdf_rivers.geometry.y

Map.add_labels(label_gdf_rivers, column='name', font_size='8pt', text_color='blue', font_weight='bold', x='x', y='y')
Map.addLayer(rivers_fc.style(color='blue', width=2), {}, 'River Paths')

Map.add_legend(title="Wheat Type", labels=["Irrigated", "Rainfed"], colors=["#00BFFF", "#FFA500"])
Map.addLayerControl()
Map


Map(center=[30.89932091766724, 49.49589754354096], controls=(WidgetControl(options=['position', 'transparent_b…